# Python for Picking up Stocks by Screening Correction

In [ ]:
import os.path
from os import path

import webbrowser

import pandas as pd
import numpy as np

import datetime
from pytz import timezone
from datetime import timedelta

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from pandas_datareader import data as pdr
import yfinance

In [ ]:
symbols = pd.read_excel('watch_list_new.xlsx')['symbol']
today = pd.Timestamp(datetime.date.today())
weekday = today.weekday()
current_time = datetime.datetime.now(timezone('EST')).time().strftime('%H:%M:%S')
date_6m_ago = today - pd.tseries.offsets.DateOffset(months=6, days=-1)

In [ ]:
stocks = {}

if path.exists("watch_list.h5") == False:    
    for symbol in symbols:
        stocks[symbol] = pdr.get_data_yahoo(symbol, start=date_6m_ago, end=today)
        stocks[symbol].to_hdf('watch_list.h5', key = symbol, mode = 'a')

else:
    for symbol in symbols:    
        stocks[symbol] = pd.read_hdf('watch_list.h5', key = symbol)

    latest_data = {}

    if weekday<5 and current_time>'06:30:00': # and current_time<'24:00:00':

        for symbol in symbols:

            last_date = stocks[symbol].index[-1]

            if last_date == today:
                new_start_date = today
                stocks[symbol] = stocks[symbol].drop([last_date])

            elif last_date.weekday()==4:
                new_start_date = last_date + timedelta(days=3)

            else:
                new_start_date = last_date + timedelta(days=1)

            latest_data[symbol] = pdr.get_data_yahoo(symbol, start=new_start_date, end=today)        
            stocks[symbol] = stocks[symbol].append(latest_data[symbol])

            stocks[symbol].to_hdf('watch_list.h5', key = symbol, mode = 'a')

In [ ]:
current_time

In [ ]:
stocks['AAPL']

In [ ]:
stocks_to_buy = pd.DataFrame(columns=['symbol',	'Date_High',	'Price_High',	'Date_Low',	'Price_Low',	'Max_Correction',	'Price_Today',	'Today_Correction',	'Rebounced'])

for symbol in symbols:
    
    highest_price = stocks[symbol]['High'].max()
    highest_date = stocks[symbol]['High'].idxmax(axis=0, skipna=True)

    lowest_price = stocks[symbol].loc[highest_date:]['Low'].min()
    lowest_date = stocks[symbol].loc[highest_date:,'Low'].idxmin(axis=0, skipna=True)

    max_correction = (lowest_price-highest_price)/highest_price

    if max_correction < -0.25:
        today_price = stocks[symbol].iloc[-1,-1]
        today_correction = (today_price-highest_price)/highest_price
        diff_today_lowest = (today_price-lowest_price)/lowest_price

        if diff_today_lowest < 0.1:
            stocks_to_buy = stocks_to_buy.append({'symbol':symbol,'Date_High':highest_date,'Price_High':highest_price,'Date_Low':lowest_date,'Price_Low':lowest_price,'Max_Correction':max_correction,'Price_Today':today_price, 'Today_Correction':today_correction,'Rebounced':diff_today_lowest},ignore_index=True)

In [ ]:
stocks_to_buy.round(4)

In [ ]:
date_index = datetime.date.today().strftime('%Y%m%d')
stocks_to_buy.round(4).to_excel('stocks_to_buy_' + date_index + '.xlsx', index=None)

In [ ]:
for sbl in stocks_to_buy['symbol']:

    url = "https://finance.yahoo.com/chart/" + sbl

    webbrowser.open_new_tab(url)